Explore use of Feature Tools to generate features

In [1]:
# ensure user defined modules are on the python path
import featuretools as ft
import os
import sys
import pandas as pd
import numpy as np

In [2]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
#print(sys.path)
from salliemae.data.experian import subset_installments, clean_account_master

In [3]:
# load tradelined data 
tradeline_df = pd.read_csv('/Users/stewarta/Documents/DATA/USA/sallie/tradelines.csv',  sep=';',low_memory= False, na_values = '""')
tradeline_df.head()

,Dir_ID,EOS_Acct,Subject_ID,Tradeline_Index,Responsible,Tradeline_Type,Date_Reported,LKTRRDTI_Data,First_Reported_Date,LKTRFDTI_Data,...,Orgnl_Grantor_Name,Orgnl_Grantor_Biz,Data_Source,Indicator_Code_1,Indicator_Code_2,Indicator_Code_3,Indicator_Code_4,Trigger_ID,Subject_Type,Unnamed: 104
0,11,8155318,41841,5,I,I,08/14/2016,8142016,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,0,B\r,NaN
1,11,8155318,41841,6,I,I,08/14/2016,8142016,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,0,B\r,NaN
2,11,8155318,41841,7,I,I,08/14/2016,8142016,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,0,B\r,NaN
3,11,8155318,41841,8,I,I,08/14/2016,8142016,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,0,B\r,NaN
4,11,8155318,41841,9,I,I,08/14/2016,8142016,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,0,B\r,NaN


In [4]:
tradeline_df = subset_installments(tradeline_df)

In [5]:
# remove columns that will not be used in the featuretools
non_featuretools = [ 'Subject_ID','Payment_History', 'Tradeline_Index']
tradeline_df.drop(non_featuretools, axis=1, inplace=True)
tradeline_df['EOS_Acct'] =  tradeline_df['EOS_Acct'].astype('int64')
tradeline_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250916 entries, 0 to 420115
Data columns (total 19 columns):
EOS_Acct               250916 non-null int64
Responsible            250916 non-null object
Tradeline_Type         250916 non-null object
Date_Reported          250916 non-null object
Subscriber_Code        250916 non-null int64
Industry_Type          250916 non-null object
Tradeline_Usage        250916 non-null object
Date_Opened            250916 non-null object
Trade_Ending_Cd        250916 non-null object
Term_Duration          250916 non-null int64
Manner_Of_Pmt_Dte      250916 non-null object
Manner_Of_Payment      250916 non-null object
History_Reviewed       250916 non-null float64
Delinquent_Payments    250916 non-null float64
Late_30_Days           250916 non-null float64
Late_60_Days           250916 non-null float64
Late_90_Days           250916 non-null float64
Indicator_Code_1       250916 non-null object
Subject_Type           250916 non-null object
dtypes: float

In [6]:
tradeline_df.head()

,EOS_Acct,Responsible,Tradeline_Type,Date_Reported,Subscriber_Code,Industry_Type,Tradeline_Usage,Date_Opened,Trade_Ending_Cd,Term_Duration,Manner_Of_Pmt_Dte,Manner_Of_Payment,History_Reviewed,Delinquent_Payments,Late_30_Days,Late_60_Days,Late_90_Days,Indicator_Code_1,Subject_Type
0,8155318,I,I,08/14/2016,1906328,VF,12,07/19/2011,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B\r
1,8155318,I,I,08/14/2016,1906328,VF,12,05/14/2012,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B\r
2,8155318,I,I,08/14/2016,1906328,VF,12,09/02/2010,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B\r
3,8155318,I,I,08/14/2016,1906328,VF,12,05/24/2013,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B\r
4,8155318,I,I,08/14/2016,1906328,VF,12,12/06/2012,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B\r


In [7]:
# remove newlines characters
tradeline_df['Subject_Type'] = [''.join(c.split()) for c in tradeline_df['Subject_Type'].astype(str)]

In [8]:
tradeline_df.head()

,EOS_Acct,Responsible,Tradeline_Type,Date_Reported,Subscriber_Code,Industry_Type,Tradeline_Usage,Date_Opened,Trade_Ending_Cd,Term_Duration,Manner_Of_Pmt_Dte,Manner_Of_Payment,History_Reviewed,Delinquent_Payments,Late_30_Days,Late_60_Days,Late_90_Days,Indicator_Code_1,Subject_Type
0,8155318,I,I,08/14/2016,1906328,VF,12,07/19/2011,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
1,8155318,I,I,08/14/2016,1906328,VF,12,05/14/2012,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
2,8155318,I,I,08/14/2016,1906328,VF,12,09/02/2010,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
3,8155318,I,I,08/14/2016,1906328,VF,12,05/24/2013,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
4,8155318,I,I,08/14/2016,1906328,VF,12,12/06/2012,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B


In [9]:
# insert index on tradelines table
tradeline_df.insert(loc=0, column='tradeline_idx', value=range(0,len(tradeline_df)))

In [10]:
tradeline_df.head()

,tradeline_idx,EOS_Acct,Responsible,Tradeline_Type,Date_Reported,Subscriber_Code,Industry_Type,Tradeline_Usage,Date_Opened,Trade_Ending_Cd,Term_Duration,Manner_Of_Pmt_Dte,Manner_Of_Payment,History_Reviewed,Delinquent_Payments,Late_30_Days,Late_60_Days,Late_90_Days,Indicator_Code_1,Subject_Type
0,0,8155318,I,I,08/14/2016,1906328,VF,12,07/19/2011,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
1,1,8155318,I,I,08/14/2016,1906328,VF,12,05/14/2012,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
2,2,8155318,I,I,08/14/2016,1906328,VF,12,09/02/2010,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
3,3,8155318,I,I,08/14/2016,1906328,VF,12,05/24/2013,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B
4,4,8155318,I,I,08/14/2016,1906328,VF,12,12/06/2012,C,36,08/01/2015,93,13.0,12.0,0.0,0.0,0.0,N,B


In [11]:
# Create entity Set
es = ft.EntitySet(id='tx')

In [12]:
es = es.entity_from_dataframe(dataframe=tradeline_df,
                         entity_id='tradelines',
                         index='tradeline_idx',
                         variable_types={#"EOS_Acct": ft.variable_types.Id, 
                                        "Responsible": ft.variable_types.Categorical, 
                                        "Tradeline_Type": ft.variable_types.Categorical,
                                        "Date_Reported": ft.variable_types.Datetime,
                                        "Subscriber_Code": ft.variable_types.Categorical,
                                        "Industry_Type": ft.variable_types.Categorical,
                                        "Tradeline_Usage": ft.variable_types.Categorical,
                                        "Date_Opened": ft.variable_types.Datetime,
                                        "Trade_Ending_Cd": ft.variable_types.Categorical,
                                        "Term_Duration": ft.variable_types.Numeric,
                                        "Manner_Of_Pmt_Dte": ft.variable_types.Datetime,
                                        "Manner_Of_Payment": ft.variable_types.Categorical,
                                        "History_Reviewed": ft.variable_types.Numeric,
                                        "Delinquent_Payments": ft.variable_types.Numeric,
                                        "Late_30_Days": ft.variable_types.Numeric,
                                        "Late_60_Days": ft.variable_types.Numeric,
                                        "Late_90_Days": ft.variable_types.Numeric,
                                        "Indicator_Code_1": ft.variable_types.Categorical,
                                        "Subject_Type": ft.variable_types.Categorical})

In [13]:
es['tradelines'].variables

[<Variable: tradeline_idx (dtype = index)>,
 <Variable: EOS_Acct (dtype = numeric)>,
 <Variable: Responsible (dtype = categorical)>,
 <Variable: Tradeline_Type (dtype = categorical)>,
 <Variable: Date_Reported (dtype: datetime, format: None)>,
 <Variable: Subscriber_Code (dtype = categorical)>,
 <Variable: Industry_Type (dtype = categorical)>,
 <Variable: Tradeline_Usage (dtype = categorical)>,
 <Variable: Date_Opened (dtype: datetime, format: None)>,
 <Variable: Trade_Ending_Cd (dtype = categorical)>,
 <Variable: Term_Duration (dtype = numeric)>,
 <Variable: Manner_Of_Pmt_Dte (dtype: datetime, format: None)>,
 <Variable: Manner_Of_Payment (dtype = categorical)>,
 <Variable: History_Reviewed (dtype = numeric)>,
 <Variable: Delinquent_Payments (dtype = numeric)>,
 <Variable: Late_30_Days (dtype = numeric)>,
 <Variable: Late_60_Days (dtype = numeric)>,
 <Variable: Late_90_Days (dtype = numeric)>,
 <Variable: Indicator_Code_1 (dtype = categorical)>,
 <Variable: Subject_Type (dtype = categ

In [14]:
# load accounts table
dtypes = {'Client_Acct': 'str'}
parse_dates = ['List_Date', 'Disp_Chg_Date']
df = pd.read_csv('/Users/stewarta/Documents/DATA/USA/sallie/accounts_master.csv', delimiter = ';', dtype=dtypes, parse_dates=parse_dates)
df.head()

,Dir_ID,EOS_Acct,Seed_Acct,Client_Acct,Client_ID,List_Date,FACS_Load_Date,Disposition,Dept,Initial_Balance,...,City,State,Zip,Experian_Score,Sungard_Score,Collector_ID,Disp_Chg_Date,Service_Date,HSG_Proscore,Unnamed: 20
0,11.0,8630398.0,8630398.0,5852500000195755,SLMFDF,2017-02-17,02/17/2017,3EMP,321.0,9692.68,...,LAS VEGAS,NV,89129,0.0,B1,SMP,2017-05-08,02/23/2011,4.0,NaN
1,11.0,8630399.0,8630399.0,5852500007564854,SLMFDF,2017-02-17,02/17/2017,9999,0.0,12942.49,...,SAINT PAUL,MN,55106,0.0,A3,0,2017-04-06,NaN,0.0,NaN
2,11.0,8630400.0,8630400.0,5852500007571503,SLMFDF,2017-02-17,02/17/2017,3GPH,321.0,5785.62,...,ALBUQUERQUE,NM,87114,0.0,B3,SMP,2017-07-21,05/28/2013,4.0,NaN
3,11.0,8630401.0,8630400.0,5852500007572162,SLMFDF,2017-02-17,02/17/2017,3GPH,321.0,468.88,...,ALBUQUERQUE,NM,87114,0.0,C2,SMP,2017-07-21,05/28/2013,0.0,NaN
4,11.0,8630402.0,8630402.0,5852500007573376,SLMFDF,2017-02-17,02/17/2017,3EMP,317.0,9394.19,...,COLORADO SPGS,CO,80903,0.0,B1,UE2,2017-07-03,05/24/2013,0.0,NaN


In [15]:
accounts_df = clean_account_master(df)

In [16]:
accounts_df.head()
accounts_non_featuretools = [ 'Dir_ID','Seed_Acct', 'Client_Acct','City', 'State', 'Zip' ]
accounts_df = accounts_df.drop(accounts_non_featuretools, axis=1, inplace=False)
accounts_df['EOS_Acct'] =  accounts_df['EOS_Acct'].astype('int64')
accounts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9573 entries, 0 to 11453
Data columns (total 13 columns):
EOS_Acct           9573 non-null int64
Client_ID          9573 non-null object
List_Date          9573 non-null datetime64[ns]
FACS_Load_Date     9573 non-null object
Disposition        9573 non-null object
Dept               9573 non-null float64
Initial_Balance    9573 non-null float64
Account_Balance    9573 non-null float64
Experian_Score     9573 non-null float64
Sungard_Score      9573 non-null object
Collector_ID       9573 non-null object
Disp_Chg_Date      9573 non-null datetime64[ns]
HSG_Proscore       9573 non-null float64
dtypes: datetime64[ns](2), float64(5), int64(1), object(5)
memory usage: 1.0+ MB


In [17]:
## '%m/%d/%Y'
es = es.entity_from_dataframe(dataframe=accounts_df,
                         entity_id='accounts',
                         index='EOS_Acct',
                         variable_types={#"EOS_Acct": ft.variable_types.Index, 
                                         "Client_ID": ft.variable_types.Categorical, 
                                        "List_Date": ft.variable_types.Datetime,
                                        "FACS_Load_Date": ft.variable_types.Datetime,
                                        "Disposition": ft.variable_types.Categorical,
                                        "Dept": ft.variable_types.Categorical,
                                        "Initial_Balance": ft.variable_types.Numeric,
                                        "Account_Balance": ft.variable_types.Categorical,
                                        "Experian_Score": ft.variable_types.Categorical,
                                        "Sungard_Score": ft.variable_types.Categorical,
                                        "Collector_ID": ft.variable_types.Categorical,
                                        "Disp_Chg_Date": ft.variable_types.Categorical,
                                        "HSG_Proscore": ft.variable_types.Categorical})
es

Entityset: tx
  Entities:
    tradelines [Rows: 250916, Columns: 20]
    accounts [Rows: 9573, Columns: 13]
  Relationships:
    No relationships

In [18]:
es['accounts'].variables

[<Variable: EOS_Acct (dtype = index)>,
 <Variable: Client_ID (dtype = categorical)>,
 <Variable: List_Date (dtype: datetime, format: None)>,
 <Variable: FACS_Load_Date (dtype: datetime, format: None)>,
 <Variable: Disposition (dtype = categorical)>,
 <Variable: Dept (dtype = categorical)>,
 <Variable: Initial_Balance (dtype = numeric)>,
 <Variable: Account_Balance (dtype = categorical)>,
 <Variable: Experian_Score (dtype = categorical)>,
 <Variable: Sungard_Score (dtype = categorical)>,
 <Variable: Collector_ID (dtype = categorical)>,
 <Variable: Disp_Chg_Date (dtype = categorical)>,
 <Variable: HSG_Proscore (dtype = categorical)>]

In [19]:
# create a new relationship
new_relationship = ft.Relationship(es["accounts"]["EOS_Acct"],
                                      es["tradelines"]["EOS_Acct"])

In [20]:
# add the relationship to the entity set
es = es.add_relationship(new_relationship)

In [21]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity="accounts")

In [24]:
feature_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9573 entries, 8147057 to 9682285
Data columns (total 97 columns):
Client_ID                                            9573 non-null object
Disposition                                          9573 non-null object
Dept                                                 9573 non-null float64
Initial_Balance                                      9573 non-null float64
Account_Balance                                      9573 non-null float64
Experian_Score                                       9573 non-null float64
Sungard_Score                                        9573 non-null object
Collector_ID                                         9573 non-null object
Disp_Chg_Date                                        9573 non-null datetime64[ns]
HSG_Proscore                                         9573 non-null float64
SUM(tradelines.Term_Duration)                        9573 non-null float64
SUM(tradelines.History_Reviewed)                     957